# Trace Exporting Example

In [ ]:
# Install zipkin python package:
# ! pip install opentelemetry-exporter-zipkin-proto-http

# Start the zipkin docker container:
# ! docker run --rm -d -p 9411:9411 --name zipkin openzipkin/zipkin

# Stop the zipkin docker container:
# ! docker stop $(docker ps -a -q --filter ancestor=openzipkin/zipkin)

In [ ]:
from opentelemetry.exporter.zipkin.proto.http import ZipkinExporter
from trulens_eval import Tru

# Need to create dummy endpont for the instrumentation of the DummyAPI to
# happen.
from trulens_eval.feedback.provider.endpoint.dummy import DummyEndpoint
from trulens_eval.tru_custom_app import TruCustomApp

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp

DummyEndpoint()

tru = Tru(
    _otel_exporter=ZipkinExporter(endpoint="http://localhost:9411/api/v2/spans")
)

tru.reset_database()

tru.start_dashboard(force=True)

In [ ]:
# Create custom app:
ca = CustomApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [ ]:
# Nested context managers.
with ta as recorder1:
    ca.llm.generate("this is the outer memory remember")
    with ta as recorder2:
        ca.memory.remember("this is the inner memory remember")
        ca.respond_to_query("hello")